<a href="https://colab.research.google.com/github/Amirrezahmi/SelfTalker/blob/main/fine-tune%20GPT2/fine_tune_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00


In [2]:
! pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 5.9 MB/s eta 0:00:00


# Fine-tune GPT-2 with our dataset

In [1]:
import torch
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    Trainer,
    TrainingArguments,
    GPT2LMHeadModel,
)

# Set the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Load the dataset
file_path = "data.txt"
# Use LineByLineTextDataset for better memory efficiency
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=file_path, block_size=128)

# Set the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)

# Set the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,  # Reduce the per-device train batch size
    logging_steps=100,
    save_steps=1000,
    prediction_loss_only=True,
    # Increase gradient accumulation steps for better GPU utilization
    gradient_accumulation_steps=8,  # Increase the gradient accumulation steps
    # Use mixed precision training to speed up the process
    fp16=True,
)

# Set the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,3.168500


TrainOutput(global_step=170, training_loss=3.15022268856273, metrics={'train_runtime': 744.3314, 'train_samples_per_second': 29.355, 'train_steps_per_second': 0.228, 'total_flos': 5052131778232320.0, 'train_loss': 3.15022268856273, 'epoch': 1.0})

# saving the model and then loading it and generating some text

In [2]:
model.save_pretrained("my_sjaved_model")

In [6]:
# Load the saved model
saved_model = GPT2LMHeadModel.from_pretrained("my_sjaved_model")

# Generate some text using the saved model
prompt = "Who is God?"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = saved_model.generate(input_ids, max_length=200, num_beams=5,no_repeat_ngram_size=2, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Who is God?

God is the Creator of the universe. He created the heavens and the earth, the sun, moon, stars, planets, and all living things. God created man in his own image, in the image of God he created him; male and female created he them. All things were created by him and for him. (Genesis 1:27–28) God is omniscient, omnipotent, all-powerful, omnibenevolent, loving, merciful, good, wise, understanding, kind, tender, compassionate, forgiving, faith-filled, faithful to one's parents, boundless in love and compassion for all creatures, generous in giving and giving generously to the poor and needy, caring for the sick and those in need, giving thanks to God for what he has done for them and their families, believing in God and his Son Jesus Christ as his only Son and Redeemer, willing to die for his name and to give his life


## Save the model to your drive, and then generate some text. I dont run this part because i've already done this.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p "/content/drive/My Drive/path/to/folder/"

In [ ]:
!cp -r /content/my_sjaved_model "/content/drive/My Drive/path/to/folder/"

load and generate

In [ ]:
# Load the saved model from Drive
saved_model = GPT2LMHeadModel.from_pretrained("/content/drive/My Drive/path/to/folder/my_sjaved_model")

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')

In [ ]:
# Generate some text using the saved model
prompt = "Who is God?"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = saved_model.generate(input_ids, max_length=200, num_beams=5,no_repeat_ngram_size=2, do_sample=True)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)